In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

%matplotlib inline

In [12]:
# import trends file, to be merged with dengue cases
trends = pd.read_csv('../../assets/complete_google_trends.csv', index_col=0)

In [13]:
trends.head()

,date,year,month,week,dengue_search,dengue_fever_search,rashes_search,vomiting_search,eye_pain_search,joint_pain_search,fever_search,wolbachia_search,headache_search,nausea_search,fogging_search
0,2014-06-01,2014,1,1,44,52,67,43,30,27,73,0,22,9,0
1,2014-13-01,2014,1,2,38,53,73,38,0,26,66,2,25,11,0
2,2014-20-01,2014,1,3,31,60,60,52,12,13,80,0,27,8,0
3,2014-27-01,2014,1,4,23,24,57,39,31,0,64,0,25,12,0
4,2014-03-02,2014,2,5,28,44,56,31,0,7,72,0,22,8,0


In [14]:
# import dengue cases file
dengue_df = pd.read_csv('../../assets/dengue.csv')

In [15]:
dengue_df.head()

,type_dengue,date,year,month,number
0,Dengue,2014-01-06,2014,1,436.0
1,DHF,2014-01-06,2014,1,1.0
2,Dengue,2014-01-13,2014,1,479.0
3,DHF,2014-01-13,2014,1,0.0
4,Dengue,2014-01-20,2014,1,401.0


In [16]:
# sum up dengue and DHF cases, as DHF is rare and is considered as a dengue case
dengue_df['dengue_sum'] = dengue_df.groupby('date')['number'].transform('sum')

In [17]:
# drop the two columns which are redundant
dengue_df = dengue_df.drop(['number', 'type_dengue'], axis=1)

In [18]:
# drop the rows with duplicate value of the sum
dengue_df = dengue_df.drop_duplicates(subset='date')

In [19]:
dengue_df.head()

,date,year,month,dengue_sum
0,2014-01-06,2014,1,437.0
2,2014-01-13,2014,1,479.0
4,2014-01-20,2014,1,401.0
6,2014-01-27,2014,1,336.0
8,2014-02-03,2014,2,234.0


In [20]:
# merge the dengue cases df with google trends df
merged_df = pd.merge(trends, dengue_df, on=['date', 'year', 'month'])

In [21]:
merged_df

,date,year,month,week,dengue_search,dengue_fever_search,rashes_search,vomiting_search,eye_pain_search,joint_pain_search,fever_search,wolbachia_search,headache_search,nausea_search,fogging_search,dengue_sum
0,2014-03-03,2014,3,9,31,54,42,30,10,14,56,2,20,10,28,187.0
1,2014-05-05,2014,5,18,18,46,47,32,12,0,69,0,24,15,0,251.0
2,2014-07-07,2014,7,27,64,100,64,24,17,20,89,0,22,7,0,891.0
3,2015-02-02,2015,2,5,18,27,42,38,0,13,56,0,27,12,0,212.0
4,2016-04-04,2016,4,13,34,50,39,46,12,8,64,0,28,13,0,291.0
5,2016-06-06,2016,6,22,27,33,34,49,9,7,71,0,27,11,19,158.0
6,2016-08-08,2016,8,31,22,20,41,41,0,0,64,0,22,14,0,197.0
7,2016-10-10,2016,10,40,27,23,46,46,15,17,66,2,29,5,0,94.0
8,2016-12-12,2016,12,49,10,0,58,60,15,12,45,0,27,10,0,81.0
